In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from tqdm import notebook
import torchvision

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
def load_digit_dataset(df):
    data = df.to_numpy(dtype=np.float32)
    X = data[:,1:]
    Y = data[:,:1]
    X = torch.tensor(X).reshape((-1, 1, 28, 28)).to(device) / 255.0
    Y = torch.tensor(Y, dtype=torch.int64).to(device)
    return X, Y

In [ ]:
# X_mnist, Y_mnist = load_digit_dataset(pd.read_csv('/kaggle/input/digit-recognizer/train.csv'))
X_kannada, Y_kannada = load_digit_dataset(pd.read_csv('/kaggle/input/Kannada-MNIST/train.csv'))
X_kannada_va, Y_kannada_va = load_digit_dataset(pd.read_csv('/kaggle/input/Kannada-MNIST/Dig-MNIST.csv'))

In [ ]:
# # https://stackoverflow.com/a/45280846/2166741
# plt.figure(figsize=(20,10))
# columns = 5
# for i in range(10):
#     plt.subplot(10 / columns + 1, columns, i + 1)
#     plt.imshow(X_mnist[i][0].cpu())

In [ ]:
# https://stackoverflow.com/a/45280846/2166741
plt.figure(figsize=(20,10))
columns = 5
for i in range(10):
    plt.subplot(10 / columns + 1, columns, i + 1)
    plt.imshow(X_kannada[i][0].cpu())

In [ ]:
# https://stackoverflow.com/a/45280846/2166741
plt.figure(figsize=(20,10))
columns = 5
for i in range(10):
    plt.subplot(10 / columns + 1, columns, i + 1)
    plt.imshow(X_kannada_va[i][0].cpu())

In [ ]:
class CustomTensorDataset(torch.utils.data.Dataset):
    """TensorDataset with support of transforms.
    """
    def __init__(self, tensors, transform=None):
        assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors)
        self.tensors = tensors
        self.transform = transform

    def __getitem__(self, index):
        x = self.tensors[0][index]

        if self.transform:
            x = self.transform(x)

        y = self.tensors[1][index]

        return x, y

    def __len__(self):
        return self.tensors[0].size(0)


In [ ]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Lambda(lambda t: t.cpu()),
    torchvision.transforms.ToPILImage(),
    torchvision.transforms.RandomAffine(translate=(0.05, 0.05), scale=(0.95,1.05), degrees=5.),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Lambda(lambda t: t.to(device)),
])

In [ ]:
dataset = CustomTensorDataset([X_kannada, Y_kannada], transform=transform)
te_va_dataset = CustomTensorDataset([X_kannada_va, Y_kannada_va], transform=transform)
tr_dataset, va_dataset = torch.utils.data.random_split(dataset, [55000, 5000])
tr_dataset = torch.utils.data.ConcatDataset([tr_dataset, te_va_dataset])

In [ ]:
dataloader = torch.utils.data.DataLoader(tr_dataset, batch_size=512, shuffle=True)
va_dataloader = torch.utils.data.DataLoader(va_dataset, batch_size=512)
te_va_dataloader = torch.utils.data.DataLoader(te_va_dataset, batch_size=512)

In [ ]:
# Fully convolutional network
model = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3), # --> 26x26
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.Dropout2d(0.1),

    nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3), # --> 24x24
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.Dropout2d(0.2),
    
    nn.MaxPool2d(kernel_size=2), # --> 12x12
    
    nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3), # --> 10x10
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.Dropout2d(0.3),
    
    nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3), # --> 8x8
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.Dropout2d(0.4),

    nn.MaxPool2d(kernel_size=2), # --> 4x4

    nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3), # --> 2x2
    nn.ReLU(),
    nn.BatchNorm2d(128),
    nn.Dropout2d(0.5),
    
    nn.AvgPool2d(kernel_size=2), # --> 1x1
    
    nn.Conv2d(in_channels=128, out_channels=10, kernel_size=1, stride=1), # --> 1x1
    nn.Flatten(),
).to(device)

In [ ]:
with torch.no_grad():
    print(model(X_kannada[:1]))

In [ ]:
# conv = list(model.modules())

In [ ]:
# # https://stackoverflow.com/a/45280846/2166741
# plt.figure(figsize=(20,10))
# columns = 5
# for i in range(10):
#     plt.subplot(10 / columns + 1, columns, i + 1)
#     plt.imshow(conv.weight.data[i][0].cpu())

In [ ]:
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lambda epoch: 0.99 ** (epoch))

In [ ]:
losses = []
va_losses = []
te_va_losses = []

In [ ]:
num_epochs = 200
for epoch in notebook.tqdm(range(num_epochs)):
    model.train()
    mean_loss = 0.
    count = 0
    for x, y in dataloader:
        count += 1
        optimizer.zero_grad()
        y_ = model(x)
        loss = criterion(y_, y.flatten())
        loss.backward()
        optimizer.step()
        mean_loss += loss.item()
        losses.append(loss.item())
    scheduler.step()
    mean_loss /= count
    print('Train Loss: {}'.format(mean_loss))
    
    with torch.no_grad():
        model.eval()
        mean_loss = 0.
        count = 0
        for x, y in va_dataloader:
            count += 1
            y_ = model(x)
            loss = criterion(y_, y.flatten())
            mean_loss += loss.item()
        mean_loss /= count
        print('Validation Loss: {}'.format(mean_loss))
        va_losses.append(mean_loss)
        
    with torch.no_grad():
        model.eval()
        mean_loss = 0.
        count = 0
        for x, y in te_va_dataloader:
            count += 1
            y_ = model(x)
            loss = criterion(y_, y.flatten())
            mean_loss += loss.item()
        mean_loss /= count
        print('Test Validation Loss: {}'.format(mean_loss))
        te_va_losses.append(mean_loss)
    
#     if epoch % 1 == 0:
#         # https://stackoverflow.com/a/45280846/2166741
#         nrows = 2
#         ncols = 5
#         plt.figure(figsize=(20, 10))
#         for j in range(10):
#             plt.subplot(nrows, ncols, j + 1)
#             plt.imshow(conv.weight.data[j][0].cpu())
#         plt.show()

In [ ]:
plt.plot(losses)

In [ ]:
plt.plot(va_losses)

In [ ]:
# # https://stackoverflow.com/a/45280846/2166741
# nrows = 2
# ncols = 5
# plt.figure(figsize=(20, 10))
# for j in range(10):
#     plt.subplot(nrows, ncols, j + 1)
#     plt.imshow(conv.weight.data[j][0].cpu())
# plt.show()

In [ ]:
X_te, _ = load_digit_dataset(pd.read_csv('/kaggle/input/Kannada-MNIST/test.csv'))

In [ ]:
with torch.no_grad():
    model.eval()
    Y_te_pred = model(X_te)

In [ ]:
Y_te_pred.shape

In [ ]:
Y_te_pred_class = Y_te_pred.argmax(dim=1)

In [ ]:
Y_te_pred_class.shape

In [ ]:
submission = pd.read_csv('/kaggle/input/Kannada-MNIST/sample_submission.csv')

In [ ]:
submission['label'] = Y_te_pred_class.cpu()

In [ ]:
submission.to_csv('submission.csv',index=False)